# Stateful Model Feedback Metrics Server
In this example we will add statistical performance metrics capabilities by levering the Seldon metrics server.

Dependencies
* Istio with a gateway set up as per docs
* Seldon Core installed
* KNative eventing v0.11.0
* KNative serving v0.11.1


In [2]:
!kubectl label namespace default knative-eventing-injection=enabled --overwrite=true

namespace/default labeled


In [8]:
!kubectl get broker

NAME      READY   REASON   URL                                               AGE
default   True             http://default-broker.default.svc.cluster.local   16m


In [4]:
%%bash
kubectl apply -f - << END
apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: multiclass-model
spec:
  predictors:
  - graph:
      children: []
      implementation: SKLEARN_SERVER
      modelUri: gs://seldon-models/sklearn/iris
      name: classifier
    name: default
    replicas: 1
END

seldondeployment.machinelearning.seldon.io/multi-class created


In [3]:
!mkdir -p config

In [23]:
%%writefile config/multiclass-service.yaml
apiVersion: serving.knative.dev/v1alpha1
kind: Service
metadata:
  name: multiclass-model-metrics-kservice
spec:
  template:
    metadata:
      annotations:
        autoscaling.knative.dev/minScale: "1"
    spec:
      containers:
      - image: seldonio/alibi-detect-server:1.3.0-dev
        imagePullPolicy: IfNotPresent
        args:
        - --model_name
        - multiclassserver
        - --http_port
        - '8080'
        - --protocol
        - seldonfeedback.http
        - --storage_uri
        - "adserver.cm_models.multiclass_one_hot.MulticlassOneHot"
        - --reply_url
        - http://message-dumper.default        
        - --event_type
        - io.seldon.serving.feedback.metrics
        - --event_source
        - io.seldon.serving.feedback
        - MetricsServer
        env:
        - name: "SELDON_DEPLOYMENT_ID"
          value: "multiclass-model"
        - name: "PREDICTIVE_UNIT_ID"
          value: "classifier"
        - name: "PREDICTIVE_UNIT_IMAGE"
          value: "alibi-detect-server:1.3.0-dev"
        - name: "PREDICTOR_ID"
          value: "default"

Overwriting config/multiclass-service.yaml


In [38]:
!kubectl apply -f config/multiclass-service.yaml

service.serving.knative.dev/multiclass-model-metrics-kservice created


In [19]:
!kubectl get kservice

NAME                                URL                                                            LATESTCREATED                             LATESTREADY   READY   REASON
multiclass-model-metrics-kservice   http://multiclass-model-metrics-kservice.default.example.com   multiclass-model-metrics-kservice-8nh2p                 False   RevisionMissing


In [5]:
%%writefile config/trigger.yaml
apiVersion: eventing.knative.dev/v1alpha1
kind: Trigger
metadata:
  name: multiclass-model-metrics-trigger
  namespace: default
spec:
  filter:
    sourceAndType:
      type: io.seldon.serving.feedback
  subscriber:
    ref:
      apiVersion: serving.knative.dev/v1alpha1
      kind: Service
      name: multiclass-model-metrics

Writing config/trigger.yaml


In [6]:
!kubectl apply -f config/trigger.yaml

trigger.eventing.knative.dev/multiclass-model-metrics-trigger created


In [9]:
!kubectl get trigger

NAME                               READY   REASON                               BROKER    SUBSCRIBER_URI   AGE
multiclass-model-metrics-trigger   False   Unable to get the Subscriber's URI   default                    25s
